In [ ]:
import torch
workDir='/usr/data/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


%cd '/usr'
!mkdir 'data'
%cd '/usr/data'


Mounted at /content/drive
/usr
/usr/data


In [ ]:
!pip install --upgrade fastai
!pip install -U aicrowd-cli

     |████████████████████████████████| 204kB 13.8MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 174kB 22.4MB/s 
     |████████████████████████████████| 61kB 10.2MB/s 
     |████████████████████████████████| 204kB 23.1MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
   

In [ ]:
API_KEY = '52ab6eb031245b7028158e2f3e993174' #Please enter your API Key from [https://www.aicrowd.com/participants/me]
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [ ]:
!aicrowd dataset download --challenge f1-car-rotation -j 3

sample_submission.csv: 100% 104k/104k [00:00<00:00, 335kB/s]
train.csv: 100% 449k/449k [00:00<00:00, 690kB/s]
test.zip:   0% 0.00/111M [00:00<?, ?B/s]
val.csv:   0% 0.00/40.9k [00:00<?, ?B/s]
val.csv: 100% 40.9k/40.9k [00:00<00:00, 278kB/s]

train.zip:   0% 0.00/444M [00:00<?, ?B/s]

val.zip:   0% 0.00/44.4M [00:00<?, ?B/s]
test.zip:  30% 33.6M/111M [00:07<00:16, 4.58MB/s]

val.zip:  76% 33.6M/44.4M [00:05<00:01, 6.06MB/s]

val.zip: 100% 44.4M/44.4M [00:06<00:00, 6.93MB/s]

train.zip:  15% 67.1M/444M [00:09<00:52, 7.19MB/s]
test.zip:  91% 101M/111M [00:21<00:02, 4.60MB/s] 
test.zip: 100% 111M/111M [00:24<00:00, 4.62MB/s]

train.zip:  38% 168M/444M [00:30<00:57, 4.79MB/s]
train.zip:  45% 201M/444M [00:34<00:42, 5.66MB/s]
train.zip:  53% 235M/444M [00:37<00:30, 6.84MB/s]
train.zip:  61% 268M/444M [00:46<00:32, 5.38MB/s]
train.zip:  68% 302M/444M [00:49<00:21, 6.44MB/s]
train.zip:  76% 336M/444M [00:53<00:15, 7.06MB/s]
train.zip:  83% 369M/444M [00:56<00:09, 7.53MB/s]
train.zip:  91% 403M

In [ ]:
!rm -rf data
!mkdir data

!unzip -q train.zip  -d data/train
!unzip -q val.zip -d data/val
!unzip -q test.zip  -d data/test

!mv train.csv data/train.csv
!mv val.csv data/val.csv
!mv sample_submission.csv data/sample_submission.csv

In [ ]:
import torch
from torch.utils.data import Dataset,DataLoader,RandomSampler
from torchvision import transforms as T
import pandas as pd
from PIL import Image

class ImageDataset(Dataset):
  def __init__(self,ImageFold,lblDict,df,transforms):
    self.ImageFold=ImageFold
    self.df=df
    self.trans=transforms
    self.lblDict=lblDict

  def __len__(self):
    return len(self.df)

  def __getitem__(self,ind):
    im=self.load_image(self.df.iloc[ind][0])
    im=self.trans(im)
    return im, self.lblDict[self.df.iloc[ind][1]]


  def load_image(self,ind):
    return Image.open(self.ImageFold+str(self.df.iloc[ind][0])+'.jpg')

In [ ]:
trainResnet=T.Compose([
#        T.Resize(imSize),
#        transforms.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])

])

lblDict={'front':0,'back':1,'right':2,'left':3}
df_train=pd.read_csv('data/train.csv')
ds_train_resnet=ImageDataset(workDir+'data/train/',lblDict,df_train,trainResnet)
dl_train_resnet=DataLoader(ds_train_resnet,batch_size=64,shuffle=True,num_workers=2)

df_val=pd.read_csv('data/val.csv')
ds_val_resnet=ImageDataset(workDir+'data/val/',lblDict,df_val,trainResnet)
dl_val_resnet=DataLoader(ds_val_resnet,batch_size=64,shuffle=False,num_workers=2)

dataloaders_dict={'train':dl_train_resnet,'val':dl_val_resnet}

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()
    val_acc_history = []
    best_acc=0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            i=0
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    i+=128

                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if(i % 8192 ==0):
                      print(loss)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_corrects += torch.sum(preds == labels.data)
                running_loss += loss.detach().item()*len(labels)
            epoch_loss = running_loss / (len(dataloaders[phase].dataset))
            epoch_acc= running_corrects / len(dataloaders[phase].dataset)
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            print('{} Loss: {:.4f}, acc:  {:.4f}'.format(phase, epoch_loss, epoch_acc))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import random

import torchvision.models as models
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

Efficient Net

In [ ]:
import torch
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

In [ ]:
model = EfficientNet.from_pretrained('efficientnet-b3',num_classes = 4)

model.to(device)
criterion=nn.CrossEntropyLoss()
num_epochs=7
optimizer =torch.optim.Adam(model.parameters(), lr=0.001)
model_ft = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_4_chel_ef_adam1_crossentropy.txt')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth



Loaded pretrained weights for efficientnet-b3
Epoch 0/6
----------
tensor(0.0409, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0460, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0010, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0103, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0542, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0092, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0218, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0013, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0871, device='cuda:0', grad_fn=<NllLossBackward>)
train Loss: 0.0549, acc:  0.9815
val Loss: 0.1860, acc:  0.9635

Epoch 1/6
----------
tensor(0.0022, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0267, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0635, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0018, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0005, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0229, devic

In [ ]:
optimizer =torch.optim.Adam(model.parameters(), lr=0.0003)
model_ft = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=5)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_4_chel_ef_adam2_entropy.txt')

Epoch 0/4
----------
tensor(9.8228e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.1138e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0001, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2901e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.0628e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0039, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.9372e-05, device='cuda:0', grad_fn=<NllLossBackward>)
train Loss: 0.0041, acc:  0.9985
val Loss: 0.0201, acc:  0.9948

Epoch 1/4
----------
tensor(4.8660e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.6810e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0193, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.8564e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0007, device='cuda:0', gra

In [ ]:
optimizer =torch.optim.Adam(model.parameters(), lr=0.00004)
model_ft = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=4)
torch.save(model.state_dict(), '/content/drive/MyDrive/weights_4_chel_ef_adam3_entropy.txt')

Epoch 0/3
----------
tensor(8.3481e-06, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3540e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.5127e-06, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.3579e-06, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.0290e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2432e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.6159e-06, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.1828e-07, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.7259e-06, device='cuda:0', grad_fn=<NllLossBackward>)
train Loss: 0.0004, acc:  0.9998
val Loss: 0.0205, acc:  0.9948

Epoch 1/3
----------
tensor(8.8103e-07, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.4997e-06, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(1.2089e-06, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(6.8479e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(5.0362e-05, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(9.78

In [ ]:
model.eval()
clsDict={0:'front',1:'back',2:'right',3:'left'}

A=[[i for i in range(10000)],['']*10000]
df=pd.DataFrame(A).transpose()
df.columns=['ImageID','label']
i=0
for f in os.listdir('data/test/'):
  im=Image.open('data/test/'+f)
  tens=torch.reshape(trainResnet(im),(1,3,256,256))
  inputs = tens.to(device)
  outputs = np.argmax(model(inputs).detach().cpu().numpy())
  df.iloc[int(f.split('.')[0]),1]=clsDict[outputs]

df.to_csv('/content/drive/MyDrive/submission.csv',index=False)

In [ ]:
!aicrowd submission create -c f1-car-rotation -f '/content/drive/MyDrive/submission.csv'

submission.csv ━━━━━━━━━━━━━━━━━━━━ 100.0% • 105.6/103.9 KB • 3.3 MB/s • 0:00:00
                                                 ╭─────────────────────────╮                                                 
                                                 │ Successfully submitted! │                                                 
                                                 ╰─────────────────────────╯                                                 
                                                       Important links                                                       
┌──────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-8/problems/f1-car-rotation/submissions/140275              │
│                  │                                                                                                        │
│  All submissions │ https://www.aicr